In [1]:
%reload_ext autoreload
%autoreload 2

## Script

### Initialize, train and test function

In [17]:
from npfd.models.base import HiddenMarkovModel

def train_evaluate(X_train, X_test, y_train, y_test, plot_results=False, 
                   log=True,
                   **kwargs):
    
    model = HiddenMarkovModel()
    
    model.initialize(X_train, **kwargs)
    model.train(X_train, y_train, **kwargs)

    model.edit([f'AT 2 4 0.2 {{ne.transP}}', 
                f'AT 4 2 0.2 {{ne.transP}}'])
    
    model.train(X_train, y_train, **kwargs)

    for i in range(1, kwargs['gaussian_duplication_times']+1):
        model.edit([f'MU {2**i} {{*.state[2-4].mix}}'])
        model.train(X_train, y_train, **kwargs)

    results = model.test(X_test, y_test, **kwargs)

    try:
        score = results['WORD_Acc']
    except:
        score = -10
    if score < 0:
        score = -10

    if log:
        run = client.set_experiment_run(desc=EXPERIMENT_DESCRIPTION)
        run.log_hyperparameters(kwargs)
        run.log_metric('Accuracy', score)
    
    if plot_results:
        if score > 70:
            viz.visualize.generate_plots(run.id, X_test, y_test, results)
        
    return results['WORD_Acc']

### Hyperparameter space search

In [13]:
X_train, X_test, y_train, y_test = data.dataset.make_dataset('dmps', 
                                                                 clean_interim_dir=True, 
                                                                 test_size=0.2,
                                                                 seed=7)

INFO:root:Converting real raw files to HTK format ...
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...


In [14]:
import skopt
from skopt.plots import plot_convergence
import matplotlib.pyplot as plt

from npfd import data

vf = [0.0001, 20] 
mv = [0.0001, 1.2] # Max is 10
wip = [0, 20]
gdt = [4, 7]

space = [skopt.space.Real(vf[0], vf[1], name='variance_floor', prior='log-uniform'),
         skopt.space.Real(mv[0], mv[1], name='minimum_variance', prior='log-uniform'),
         skopt.space.Real(wip[0], wip[1], name='word_insertion_penalty', prior='uniform'),
         skopt.space.Integer(gdt[0], gdt[1], name='gaussian_duplication_times')]

@skopt.utils.use_named_args(space)
def objective(**params):
    return -1.0 * train_evaluate(X_train, X_test, y_train, y_test, **params)

### Set up VertaAI

In [15]:
from verta import Client

HOST = 'http://127.0.0.1:3000'
PROJECT_NAME = "NPF Detector"
EXPERIMENT_NAME = "26/07/2021 - Automated search - Seed"
EXPERIMENT_DESCRIPTION = f"""
* Make dataset in each iteration

Search space :
- minimum_variance={mv[0], mv[1]}
- word_insertion_penalty={wip[0], wip[1]}
- gaussian_duplication_times= {gdt[0], gdt[1]}""" 

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

connection successfully established
set existing Project: NPF Detector from personal workspace
set existing Experiment: 26/07/2021 - Automated search - Seed


Run the optimizer

In [18]:
results = skopt.gp_minimize(objective, space, n_calls=200)

best_auc = -1.0 * results.fun
best_params = results.X

INFO:root:Initializing model...
INFO:root:/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc
INFO:root:Training the model...



/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks/hyperparameter-search
Pruning-Off

Pruning-Off



INFO:root:Editing model 3
INFO:root:Training the model...


Pruning-Off


Pruning-Off

Pruning-Off



INFO:root:Editing model 7
INFO:root:Training the model...


...

INFO:root:Editing model 19
INFO:root:Training the model...


Pruning-Off


Pruning-Off

Pruning-Off



INFO:root:Editing model 23
INFO:root:Training the model...


Pruning-Off




KeyboardInterrupt: 

In [8]:
# print("""Best parameters:
# - variance_floor=%.6f
# - minimum_variance=%.6f
# - word_insertion_penalty=%.2f
# - grammar_scale_factor=%.2f
# - gaussian_duplication_times=%d""" % (results.x[0], results.x[1],
#                                       results.x[2], results.x[3],
#                                       results.x[4]))

plot_convergence(results)
plt.show()

skopt.plots.plot_evaluations(results)
plt.show()

NameError: name 'results' is not defined

In [25]:
results#.get_hyperparameters()

          fun: -89.7
    func_vals: array([-69.19, -47.4 , -60.61, -25.16, -84.94, -62.79, -55.83, -54.65,
       -68.29,  -6.54, -86.42, -74.12, -83.53, -75.14, -74.39, -47.13,
       -58.43, -73.01, -56.62,  80.  , -76.19, -72.83,  -9.47, -82.42,
       -80.84, -78.21, -68.49, -45.03, -81.94, -83.24, -78.17,  -7.56,
       -77.78, -55.43, -72.  , -78.57, -76.79, -73.71, -79.04, -81.67,
       -69.8 , -88.82, -69.72, -70.33, -80.22, -45.45, -52.33, -50.89,
       -61.05, -74.27, -66.27, -15.61, -74.16, -51.85, -59.17,  12.16,
       -74.71, -75.92, -30.86, -72.61, -42.18, -71.88,  -1.16, -74.33,
       -79.89, -89.7 , -83.23, -85.63, -86.55, -83.14, -85.89, -82.39,
       -84.05, -83.33, -83.04, -78.18, -81.76, -78.16, -76.13, -70.78,
       -77.33, -71.58, -77.09, -83.96, -73.77, -76.13, -83.33, -80.81,
       -77.62, -78.82, -78.02, -69.48, -69.75, -76.36, -69.13, -84.31,
       -84.02, -63.64, -79.73, -79.44, -74.47, -68.53, -78.95, -76.92,
       -79.63, -80.95, -78.36, -76.47, -8

## Best experiments

In [116]:
print(f"# de experimentos: {expt.expt_runs}")

for e in expt.expt_runs.find("metrics.Accuracy > 70"):
    params = e.get_hyperparameters()
    train_evaluate(X_train, X_test, y_train, y_test, log=False, **params)

# de experimentos: <ExperimentRuns containing 324 runs>
set existing ExperimentRun: Run 1785816255236353315084


INFO:root:Converting real raw files to HTK format ...
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...
INFO:root:Initializing model...
INFO:root:/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc
INFO:root:Training the model...



/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks/hyperparameter-search
Pruning-On[250.0 150.0 1000.0]



INFO:root:Most trained model: 3
INFO:root:Editing model 3


Pruning-On[250.0 150.0 1000.0]

Pruning-On[250.0 150.0 1000.0]



...

INFO:root:Most trained model: 35
INFO:root:Editing model 35
INFO:root:Most trained model: 36
INFO:root:Training the model...


Pruning-On[250.0 150.0 1000.0]


Pruning-On[250.0 150.0 1000.0]

Pruning-On[250.0 150.0 1000.0]



INFO:root:Most trained model: 39
INFO:root:Testing model: 39


Pruning-On[250.0 150.0 1000.0]

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/config -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/39/macros -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/39/hmmdefs -p 1.5116202479 -s 50.5001414153 -A -T 0 -S /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A.scp -i /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf -w /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/wdnet /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/dict /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Mon Jul  5 20:08:13 2021
  Ref : >ocumentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_labels.mlf
  Rec : >gfogwil/Documentos/Facultad/Te

UnboundLocalError: local variable 'run' referenced before assignment